# TPz lite

_Authors: Andreia Dourado, Bruno Moraes_

_Adapted from Sam Schmidt example notebook: https://github.com/LSSTDESC/rail_tpz ._


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import rail
import qp
from rail.core.data import TableHandle, PqHandle, ModelHandle, QPHandle, DataHandle, Hdf5Handle
from rail.core.data import TableHandle
from rail.core.stage import RailStage

### 1. Reading the data

In [2]:
data = pd.read_csv("/home/andreia.dourado/ic-photoz/andreia_dourado/training_set_random_NaN.csv")

In [3]:
data

,Unnamed: 0,coord_ra,coord_dec,redshift,mag_u,mag_g,mag_r,mag_i,mag_z,mag_y,magerr_u,magerr_g,magerr_r,magerr_i,magerr_z,magerr_y
0,1,49.896582,-44.611338,1.018660,23.313167,23.441160,23.382988,23.083580,22.702812,22.585739,0.052674,0.019568,0.022611,0.031493,0.053542,0.100031
1,2,49.932013,-44.612139,1.060460,22.467428,22.629194,22.603159,22.405534,22.178428,21.909421,0.025942,0.009869,0.011685,0.019712,0.033545,0.062238
2,21,49.902982,-44.476808,1.546170,24.435287,24.272955,24.306442,23.910042,23.656964,NaN,0.111388,0.038465,0.046264,0.055356,0.145509,NaN
3,24,49.974539,-44.508610,0.678071,23.829824,23.200197,22.279909,21.414007,21.164629,20.767440,0.075128,0.013808,0.007567,0.006564,0.012591,0.017474
4,25,49.921902,-44.578027,0.764488,22.824555,22.774506,22.244276,21.376655,20.965122,20.733605,0.039484,0.012208,0.009455,0.008473,0.014115,0.022103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43309,457197,58.706380,-27.121380,0.744162,23.917941,23.806971,23.265423,22.487238,22.316108,22.087569,0.069663,0.017435,0.013065,0.010838,0.023149,0.040792
43310,457243,58.597580,-27.143728,1.352260,22.503866,22.478124,22.551313,22.483252,22.406050,21.881320,0.019913,0.005633,0.006828,0.010920,0.024045,0.034610
43311,457254,58.606894,-27.013599,0.858400,24.288492,24.009955,23.156547,22.086376,21.667211,21.527968,0.155776,0.035817,0.021822,0.014443,0.023314,0.035331
43312,457256,58.586839,-27.031181,0.633373,21.647205,21.512366,21.092169,20.557911,20.446284,20.225013,0.025902,0.005801,0.005269,0.005269,0.010528,0.016716


### 3. Run TPz

In [116]:
DS = RailStage.data_store
DS.__class__.allow_overwrite = True

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import os
import tables_io

In [6]:
from rail.estimation.algos.tpz_lite import TPZliteInformer

Train: 70%

In [7]:
fraction = int(0.7 * len(data))
fraction

30319

In [8]:
training_csv = data.sample(fraction,random_state=40)

In [9]:
training_csv


,Unnamed: 0,coord_ra,coord_dec,redshift,mag_u,mag_g,mag_r,mag_i,mag_z,mag_y,magerr_u,magerr_g,magerr_r,magerr_i,magerr_z,magerr_y
42827,435279,60.704527,-27.495874,2.090530,23.597365,23.581757,23.822283,24.058855,24.344960,NaN,0.041069,0.013178,0.016167,0.037777,0.118259,NaN
41149,416658,58.906838,-28.226691,0.873823,22.935931,23.126745,22.811510,22.199160,22.045821,21.822639,0.027839,0.009717,0.008731,0.008641,0.022305,0.032146
42289,429391,61.200243,-28.656315,0.627421,24.795155,23.998178,22.832508,22.073154,21.867396,21.700125,0.098684,0.017129,0.007226,0.006325,0.014552,0.022227
39269,396151,59.624008,-30.666498,1.129980,24.398873,24.686644,24.218548,24.011434,23.312765,23.267361,0.107718,0.049499,0.038373,0.057907,0.082246,0.158905
20676,205307,58.687785,-37.467142,1.031710,22.132586,22.286778,22.347926,22.185000,21.805518,21.683204,0.009398,0.004003,0.004786,0.006145,0.013461,0.021452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24077,240918,50.669692,-35.151908,1.439470,23.746549,23.737261,23.791611,23.726896,23.724539,23.025461,0.105304,0.041528,0.037130,0.057230,0.137216,0.141746
6426,62387,60.581124,-42.798440,1.000960,25.449305,25.620892,24.791998,23.878453,23.194306,22.771912,0.178233,0.072878,0.032639,0.028979,0.040917,0.061724
36957,371746,55.471531,-30.414963,0.987359,24.169206,24.333225,23.982075,23.435304,22.916391,22.923781,0.068171,0.027874,0.020764,0.022422,0.039193,0.077550
22504,225118,54.048322,-35.607546,0.601097,24.684034,24.379624,23.501109,22.961309,22.882711,22.725969,0.106186,0.028526,0.014726,0.015600,0.040645,0.062306


To write the hdf5 file correctly, it was converted the dataframe to a dictionary with parameter _"orient='list'"_, so that _'list': dict like {column->[values]}_

Creating the dictionary:

In [11]:
dict_list = training_csv.to_dict(orient='list')

In [12]:
hdf5_file_path = ('/home/andreia.dourado/ic-photoz/andreia_dourado/training_data_24_1_NaN.hdf5')

Writing the dictionary values in the hdf5 file:

In [13]:
import h5py

In [14]:
with h5py.File(hdf5_file_path, 'w') as hdf5_file:
    def write_dict(group, data):
        for key, value in data.items():
            if isinstance(value, dict):
                subgroup = group.create_group(str(key))
                write_dict(subgroup, value)
            else:
                group[str(key)] = value

    write_dict(hdf5_file, dict_list)

In [15]:
datafile = ('/home/andreia.dourado/ic-photoz/andreia_dourado/training_data_24_1_NaN.hdf5')

Creating the training_data file:

In [16]:
training_data = DS.read_file("training_data", TableHandle, datafile)

In [17]:
print(training_data.data)

OrderedDict({'Unnamed: 0': array([435279, 416658, 429391, ..., 371746, 225118,  72568]), 'coord_dec': array([-27.4958739, -28.2266911, -28.6563149, ..., -30.4149631,
       -35.6075464, -41.3970488]), 'coord_ra': array([60.7045266, 58.9068381, 61.2002426, ..., 55.4715305, 54.048322 ,
       53.7107677]), 'mag_g': array([23.58175709, 23.12674472, 23.99817816, ..., 24.33322539,
       24.37962365, 24.97125148]), 'mag_i': array([24.05885451, 22.19915986, 22.07315404, ..., 23.43530424,
       22.96130907, 23.95305654]), 'mag_r': array([23.82228302, 22.81151031, 22.83250827, ..., 23.98207505,
       23.50110903, 24.63072543]), 'mag_u': array([23.59736522, 22.93593084, 24.79515467, ..., 24.16920572,
       24.68403428, 24.85583339]), 'mag_y': array([        nan, 21.82263947, 21.70012511, ..., 22.9237805 ,
       22.72596857, 23.38613609]), 'mag_z': array([24.34495979, 22.04582064, 21.86739564, ..., 22.91639079,
       22.88271141, 23.82356645]), 'magerr_g': array([0.01317753, 0.00971743, 0.0

In [19]:
len(training_data.data['coord_ra'])

30319

#### 3.1 Setting the parameters

In [21]:
bands = ["g", "i", "r", "u", "y", "z"]
new_err_dict = {}
train_atts = []
new_mag_limits={}
for band in bands:
    train_atts.append(f"mag_{band}")
    new_err_dict[f"mag_{band}"] = f"magerr_{band}"
    new_mag_limits[f"mag_{band}"] = max(training_data.data[f"mag_{band}"])
# redshift is also an attribute used in the training, but it does not have an associated
# error its entry in the err_dict should be set to "None"
new_err_dict["redshift"] = None

print(new_err_dict)
print(train_atts)
print(new_mag_limits)

{'mag_g': 'magerr_g', 'mag_i': 'magerr_i', 'mag_r': 'magerr_r', 'mag_u': 'magerr_u', 'mag_y': 'magerr_y', 'mag_z': 'magerr_z', 'redshift': None}
['mag_g', 'mag_i', 'mag_r', 'mag_u', 'mag_y', 'mag_z']
{'mag_g': 26.16050063527215, 'mag_i': 24.09996066356848, 'mag_r': 25.671546849865507, 'mag_u': 25.758896062909223, 'mag_y': nan, 'mag_z': 24.60330390402684}


In [22]:
new_err_dict.items()

dict_items([('mag_g', 'magerr_g'), ('mag_i', 'magerr_i'), ('mag_r', 'magerr_r'), ('mag_u', 'magerr_u'), ('mag_y', 'magerr_y'), ('mag_z', 'magerr_z'), ('redshift', None)])

In [113]:
tpz_dict = dict(zmin=min(training_data.data['redshift']), 
                zmax=max(training_data.data['redshift']), 
                nzbins=301, 
                mag_limits=new_mag_limits,
                bands=train_atts,
                hdf5_groupname=None,
                use_atts=train_atts,
                err_dict=new_err_dict,
                nrandom=1, 
                ntrees=1,
                tree_strategy='sklearn') 

#### 3.2 Inform method

In [114]:
pz_train = TPZliteInformer.make_stage(name='inform_TPZ', model='/home/andreia.dourado/estimator_skl_run1_mag_lim_24_1_NaN.pkl', **tpz_dict)

In [115]:
%%time
pz_train.inform(training_data)

self._parallel is serial, number of processors we will use is 1
using sklearn decision trees
making a total of 1 trees for 1 random realizations * 1 bootstraps
0 - 1 -------------> to core 0
making 1 of 1...
Inserting handle into data store.  model_inform_TPZ: /home/andreia.dourado/inprogress_estimator_skl_run1_mag_lim_24_1_NaN.pkl, inform_TPZ
CPU times: user 269 ms, sys: 6.02 ms, total: 275 ms
Wall time: 283 ms


#### 3.3 Estimate stage

Selecting the data:

In [29]:
validation= data.drop(training_csv.index)

In [30]:
validation

,Unnamed: 0,coord_ra,coord_dec,redshift,mag_u,mag_g,mag_r,mag_i,mag_z,mag_y,magerr_u,magerr_g,magerr_r,magerr_i,magerr_z,magerr_y
0,1,49.896582,-44.611338,1.018660,23.313167,23.441160,23.382988,23.083580,22.702812,22.585739,0.052674,0.019568,0.022611,0.031493,0.053542,0.100031
7,30,49.989465,-44.487558,0.998367,24.354313,23.914351,23.196309,22.398616,21.764163,21.609353,0.146997,0.032126,0.020101,0.019654,0.027102,0.041615
9,68,50.155601,-44.577740,2.755400,24.757747,23.873206,23.721297,23.665739,23.718594,NaN,0.162807,0.022138,0.023144,0.042640,0.117303,NaN
11,94,50.014969,-44.627603,0.305376,23.439910,22.478995,21.648994,21.297372,21.167564,21.329991,0.112644,0.013909,0.007365,0.010462,0.020075,0.034824
24,157,50.557608,-44.604225,1.720570,24.007413,24.082031,23.988881,23.825450,23.772340,NaN,0.091267,0.033634,0.035825,0.058230,0.160174,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43291,456982,58.572009,-27.256989,2.110160,24.179848,23.717986,23.654261,23.627721,23.797871,23.666051,0.090114,0.014303,0.016626,0.027543,0.078384,0.148777
43294,457006,58.521828,-27.318691,1.549690,23.594232,23.449200,23.457782,23.223035,23.001764,22.590730,0.042497,0.009419,0.012361,0.016072,0.031674,0.050439
43301,457106,58.512239,-27.435490,0.822266,24.757646,24.238197,23.652315,22.661399,22.407881,22.374565,0.162296,0.024978,0.018104,0.012948,0.025098,0.049746
43303,457151,58.537537,-27.471764,1.863610,24.748354,24.206069,23.993209,23.975910,23.651158,23.725810,0.149846,0.027861,0.026816,0.046688,0.089240,0.194381


Writing hdf5, similary to training set:

In [31]:
dict_list = validation.to_dict(orient='list')

In [32]:
hdf5_file_path = ('/home/andreia.dourado/ic-photoz/andreia_dourado/test_data_24_1_NaN.hdf5')

In [33]:
with h5py.File(hdf5_file_path, 'w') as hdf5_file:
    def write_dict(group, data):
        for key, value in data.items():
            if isinstance(value, dict):
                subgroup = group.create_group(str(key))
                write_dict(subgroup, value)
            else:
                group[str(key)] = value

    write_dict(hdf5_file, dict_list)

Creating the test_data file:

In [34]:
testfile = ('/home/andreia.dourado/ic-photoz/andreia_dourado/test_data_24_1_NaN.hdf5')

In [35]:
test_data = DS.read_file("test_data", TableHandle, testfile)

In [36]:
test_data.data

OrderedDict([('Unnamed: 0',
              array([     1,     30,     68, ..., 457106, 457151, 457197])),
             ('coord_dec',
              array([-44.6113379, -44.4875581, -44.5777398, ..., -27.4354898,
                     -27.4717645, -27.1213796])),
             ('coord_ra',
              array([49.8965823, 49.9894646, 50.1556014, ..., 58.5122386, 58.5375367,
                     58.7063801])),
             ('mag_g',
              array([23.44115999, 23.914351  , 23.87320605, ..., 24.2381967 ,
                     24.2060689 , 23.80697121])),
             ('mag_i',
              array([23.08357987, 22.39861617, 23.66573928, ..., 22.66139903,
                     23.97591036, 22.48723822])),
             ('mag_r',
              array([23.38298767, 23.19630946, 23.7212968 , ..., 23.65231547,
                     23.99320874, 23.26542341])),
             ('mag_u',
              array([23.31316723, 24.35431315, 24.75774728, ..., 24.75764604,
                     24.74835364, 23.9

Run:

In [ ]:
import pickle

In [ ]:
model = pickle.load(open('/home/andreia.dourado/ic-photoz/andreia_dourado/estimator_tpz_run_102trees.pkl','rb'))

In [37]:
from rail.estimation.algos.tpz_lite import TPZliteEstimator

In [38]:
test_dict = dict(hdf5_groupname=None,
                test_err_dict=new_err_dict,
                mag_limits=new_mag_limits)
test_dict

{'hdf5_groupname': None,
 'test_err_dict': {'mag_g': 'magerr_g',
  'mag_i': 'magerr_i',
  'mag_r': 'magerr_r',
  'mag_u': 'magerr_u',
  'mag_y': 'magerr_y',
  'mag_z': 'magerr_z',
  'redshift': None},
 'mag_limits': {'mag_g': 26.16050063527215,
  'mag_i': 24.09996066356848,
  'mag_r': 25.671546849865507,
  'mag_u': 25.758896062909223,
  'mag_y': nan,
  'mag_z': 24.60330390402684}}

In [109]:
model = pz_train.get_handle('model')

In [110]:
model

In [111]:
test_runner = TPZliteEstimator.make_stage(name="test_tpz", output="output_run_skl_NaN.hdf5",
                                          model=model, **test_dict)

In [112]:
%%time
results = test_runner.estimate(test_data)

Process 0 running estimator on chunk 0 - 10000
loading 100 total trees from model


/opt/conda/envs/[env]/lib/python3.12/site-packages/rail/estimation/algos/mlz_utils/utils_mlz.py:127: RuntimeWarning: divide by zero encountered in scalar divide
  ef = ((e68 - e681) / (area - area1)) * (0.68 - area1) + e681


Inserting handle into data store.  output_test_tpz: inprogress_output_run_skl_NaN.hdf5, test_tpz
Process 0 running estimator on chunk 10000 - 12995
loading 100 total trees from model
CPU times: user 4min 29s, sys: 1.01 s, total: 4min 30s
Wall time: 4min 29s
